In [1]:
import pandas as pd
import unicodedata
import numpy as np
import os

In [2]:


def leer_nombres_archivos():
    """Obtiene una lista de archivos en el directorio actual"""
    archivos = [archivo for archivo in os.listdir(os.getcwd())]
    return archivos

def tratar_texto(texto):
    """Normaliza cadenas eliminando tildes y caracteres especiales"""
    if isinstance(texto, str):
        texto_normalizado = unicodedata.normalize('NFD', texto)
        texto_sin_tildes = ''.join(c for c in texto_normalizado if unicodedata.category(c) != 'Mn')
        return texto_sin_tildes    
    else:
        return texto  # Si no es texto, devuelve el valor original

def diccionario_archivos_excel():
    """Crea un diccionario con los datos de los archivos Excel procesados"""
    archivos_excel = {}
    for archivo in leer_nombres_archivos():
        # Solo procesar archivos con extensión .xlsx
        if archivo.endswith(".xlsx"):
            ruta_completa = os.path.join(os.getcwd(), archivo)
            try:
                # Cargar el archivo Excel
                df = pd.read_excel(
                    ruta_completa,
                    skiprows=7,
                    usecols="D:K",
                    engine="openpyxl"
                )
                # Renombrar columnas específicas
                df = df.rename(columns={
                    "COMUNIDAD AUTÓNOMA": "COMUNIDAD_AUTONOMA",
                    "FECHA POSESIÓN": "FECHA_POSESION",
                    "FECHA BAJA": "FECHA_BAJA",
                    "CÓDIGO INE": "CODIGO_NIE"
                })
                # Convertir fechas a formato datetime sin horas, minutos ni segundos
                

                # Aplicar tratamiento de texto a cada celda
                df_sin_tildes = df.map(tratar_texto)
                # Guardar el DataFrame en el diccionario
                nombre_base = os.path.splitext(archivo)[0]
                archivos_excel[nombre_base] = df_sin_tildes
                print(f"Datos del archivo {archivo} cargados con éxito.")
            except Exception as e:
                print(f"Error al procesar el archivo {archivo}: {e}")
    return archivos_excel



# Procesar los archivos Excel
archivos_excel = diccionario_archivos_excel()

# Guardar los resultados en un único archivo Excel
archivo_salida = "excel_completo.xlsx"
with pd.ExcelWriter(archivo_salida, engine="openpyxl") as writer:
    for nombre_hoja, df in archivos_excel.items():
        # Guardar cada DataFrame en una hoja separada
        df.to_excel(writer, sheet_name=nombre_hoja, index=False)

print(f"Archivo combinado guardado como {archivo_salida}")


Datos del archivo Alcaldes_Mandato_1979_1983.xlsx cargados con éxito.
Datos del archivo Alcaldes_Mandato_1983_1987.xlsx cargados con éxito.
Datos del archivo Alcaldes_Mandato_1987_1991.xlsx cargados con éxito.
Datos del archivo Alcaldes_Mandato_1991_1995.xlsx cargados con éxito.
Datos del archivo Alcaldes_Mandato_1995_1999.xlsx cargados con éxito.
Datos del archivo Alcaldes_Mandato_1999_2003.xlsx cargados con éxito.
Datos del archivo Alcaldes_Mandato_2003_2007.xlsx cargados con éxito.
Datos del archivo Alcaldes_Mandato_2007_2011.xlsx cargados con éxito.
Datos del archivo Alcaldes_Mandato_2011_2015.xlsx cargados con éxito.
Datos del archivo Alcaldes_Mandato_2015_2019.xlsx cargados con éxito.
Datos del archivo Alcaldes_Mandato_2019_2023.xlsx cargados con éxito.
Archivo combinado guardado como excel_completo.xlsx


In [3]:
ruta_archivo = "excel_completo.xlsx"
hojas = pd.read_excel(ruta_archivo, sheet_name=None)
    
# Combinar datos de todas las hojas en un solo DataFrame
df_legislaturas = pd.concat(hojas.values(), ignore_index=True)

# Para tema de formateo de fechas para las tablas FECHA_POSESION y FECHA_BAJA
df_legislaturas["FECHA_POSESION"] = pd.to_datetime(df_legislaturas["FECHA_POSESION"], errors="coerce", dayfirst=True).dt.date
df_legislaturas["FECHA_BAJA"] = pd.to_datetime(df_legislaturas["FECHA_BAJA"], errors="coerce", dayfirst=True).dt.date

# Contar cuántas veces aparece cada alcalde
conteo_alcaldes = df_legislaturas.groupby("NOMBRE").size()

# Filtrar solo los alcaldes que aparecen más de una vez
alcaldes_repetidos = conteo_alcaldes[conteo_alcaldes > 1].index

# Filtrar el DataFrame original con estos alcaldes y se usará de base
df_repetidos = df_legislaturas[df_legislaturas["NOMBRE"].isin(alcaldes_repetidos)]

# Agrupamos por nombre en una columna y egrContar cuántas veces ha sido elegido cada alcalde 
df_legislaturas_repetidos = df_repetidos.groupby(["NOMBRE","MUNICIPIO"])["FECHA_POSESION"].apply(list).reset_index()
df_legislaturas_repetidos["REELEGIDO"] = df_legislaturas_repetidos["FECHA_POSESION"].apply(len)
df_legislaturas_repetidos = df_legislaturas_repetidos[df_legislaturas_repetidos["REELEGIDO"] > 1]

df_legislaturas_repetidos.sort_values(by="REELEGIDO",ascending=False)

,NOMBRE,MUNICIPIO,FECHA_POSESION,REELEGIDO
26466,"ZARAGOZA ALCOVER, VICENTE",CASTILLEJA DEL CAMPO,"[1979-04-21, 1983-05-27, 1987-06-30, 1991-06-1...",7
3366,"BLANCO ZUBIETA, ROBERTO",RIBAMONTAN AL MONTE,"[1979-04-19, 1983-05-23, 1987-06-30, 1991-06-1...",7
20334,"OREJAS GONZALEZ, MIGUEL",VALDELUGUEROS,"[1979-04-19, 1983-05-24, 1987-06-30, 1991-06-1...",6
20393,"ORTI MARTINEZ, JOAQUIN",ONTUR,"[1979-04-19, 1983-05-23, 1987-06-30, 1991-06-1...",6
4627,"CENTELLES NEBOT, GONZALO",COSTUR,"[1979-04-19, 1983-05-23, 1987-06-30, 1991-06-1...",6
...,...,...,...,...
26479,"ZARZA MATEOS, ANASTASIO",TORNO (EL),"[1987-06-30, 1991-06-15]",2
26477,"ZARZA HERRAEZ, DOMICIANO",MONSALUPE,"[1983-11-21, 1987-06-30]",2
27,ABEL CELSO GARCIA MARTIN,San Cristobal de la Cuesta,"[2015-06-13, 2019-06-15]",2
26,"ABEJON ARRIBAS, FAUSTINO",PINILLA-TRASMONTE,"[1987-11-28, 1999-07-03]",2


2.En esta burbuja vamos a saber lo siguiente:

De repetir mandato. Qué partidos políticos tiveron máis éxito? Qué alcaldes mudaron de partido?

In [ ]:
partidos_exitosos = df_repetidos.groupby("LISTA")["NOMBRE"].count().reset_index() 
partidos_exitosos.columns = ["LISTA", "VECES_VOTADOS"]

partidos_exitosos = partidos_exitosos.sort_values(by="VECES_VOTADOS", ascending=False)
print(partidos_exitosos)

alcaldes_cambiantes = df_repetidos.groupby("NOMBRE")["LISTA"].nunique().reset_index()
alcaldes_cambiantes["REELEGIDO"] = df_repetidos["LISTA"]
alcaldes_cambiantes.columns = ["NOMBRE", "PARTIDOS_DISTINTOS","REELEGIDO"]

alcaldes_cambiantes = alcaldes_cambiantes[alcaldes_cambiantes["PARTIDOS_DISTINTOS"] > 1]
alcaldes_cambiantes = alcaldes_cambiantes.sort_values(by="PARTIDOS_DISTINTOS", ascending=False)
print(alcaldes_cambiantes)

      LISTA  VECES_VOTADOS
76     PSOE          19264
68       PP          18405
50      IND           3457
27      CIU           2600
55    OTROS           1646
..      ...            ...
46  GANEMOS              1
32       CS              1
69     PPSO              1
74      PSG              1
86     UPSA              1

[95 rows x 2 columns]
                               NOMBRE  PARTIDOS_DISTINTOS REELEGIDO
5559       FERNANDEZ FERNANDEZ, PEDRO                   7       NaN
16802              PEREZ PEREZ, JESUS                   7      PSOE
14339        MARTINEZ RODRIGUEZ, JOSE                   6       NaN
18828          SANCHEZ ALEGRIA, CESAR                   6        AP
20824    VILLARINO RODRIGUEZ, ALFONSO                   5       NaN
...                               ...                 ...       ...
20960  ZABALZA ZABALEGUI, Mª SAGRARIO                   2      PSOE
20958      ZABALETA MORIONES, ISIDORO                   2    UC-CDS
20955               YUSTE YUSTE, LUIS    

3.En esta burbuja vamos a calular lo siguiente:

Cantos alcaldes cesaron antes de tempo?

In [ ]:
df_cesados = df_legislaturas[df_legislaturas["FECHA_BAJA"].notna()]
df_cesados_alcaldes_totales = df_cesados.groupby("FECHA_BAJA")["NOMBRE"].unique().count()


print(f"Número de alcaldes que se fueron antes de tiempo: {df_cesados_alcaldes_totales}")

Número de alcaldes que se fueron antes de tiempo: 4698


4.En esta burbuja vamos a conseguir lo siguiente:

Cal é o soldo máis alto e de que alcaldes?

In [ ]:

ruta_archivo_alcaldes = "Alcaldes_Mandato_2019_2023.xlsx"
ruta_archivo_retribuciones = "tabula-retribuciones_alcaldes_2021.csv"
df_alcaldes = pd.read_excel(
    ruta_archivo_alcaldes,
    skiprows=7,
    usecols="D:K",
    engine="openpyxl"
)
df_alcaldes = df_alcaldes.rename(columns={
    "COMUNIDAD AUTÓNOMA": "COMUNIDAD_AUTONOMA",
    "FECHA POSESIÓN": "FECHA_POSESION",
    "FECHA BAJA": "FECHA_BAJA",
    "CÓDIGO INE": "CODIGO_NIE"
})
df_alcaldes = df_alcaldes.map(tratar_texto)
df_retribuciones = pd.read_csv(ruta_archivo_retribuciones, skiprows=1)
df_retribuciones = df_retribuciones.rename(columns={"CCAA": "COMUNIDAD_AUTONOMA",
                                                    "DEDICACIÓN": "REGIMEN_DEDICACION",
                                                    "PERCIBIDO": "TOTAL_PERCIBIDO",
                                                    "AYUNTAMIENTO": "MUNICIPIO"
                                                    }
                                           )
df_retribuciones = df_retribuciones.map(tratar_texto)
df_retribuciones_alcaldes = pd.merge(df_alcaldes, df_retribuciones, on="MUNICIPIO", how="inner")

df_retribuciones_alcaldes["TOTAL_PERCIBIDO"] = df_retribuciones_alcaldes["TOTAL_PERCIBIDO"].astype(str).str.replace(".","").str.replace(",",".").astype(float)

df_retribuciones_alcaldes = df_retribuciones_alcaldes[["TOTAL_PERCIBIDO","NOMBRE","MUNICIPIO","COMUNIDAD_AUTONOMA_x","LISTA"]]
df_retribuciones_alcaldes.sort_values(by="TOTAL_PERCIBIDO", ascending=False).head(10)

,TOTAL_PERCIBIDO,NOMBRE,MUNICIPIO,COMUNIDAD_AUTONOMA_x,LISTA
6278,108517.80,Jose Luis Martinez-Almeida Navasques,Madrid,"Madrid, Comunidad de",PP
4288,100000.04,ADA COLAU BALLANO,Barcelona,Cataluna,C. ELECTORAL
6771,98018.40,JUAN MARIA ABURTO RIKE,Bilbao,Pais Vasco,EAJ-PNV
705,88673.68,ANTONIO MUNOZ MARTINEZ,Sevilla,Andalucia,PSOE
704,88673.68,JUAN ESPADAS CEJAS,Sevilla,Andalucia,PSOE
6793,83985.57,AMAIA AGIRRE MUNOA,Getxo,Pais Vasco,EAJ-PNV
13,83376.86,MARIA DEL MAR VAZQUEZ AGUERO,Almeria,Andalucia,PP
12,83376.86,RAMON FERNANDEZ-PACHECO MONTERREAL,Almeria,Andalucia,PP
1474,83054.37,JOSE JULIAN MENA PEREZ,Arona,Canarias,PSOE
591,82314.82,Francisco Manuel de la Torre Prados,Malaga,Andalucia,PP


5.En esta burbuja vamos a conseguir lo siguiente:

Relaciona os soldos altos e baixos coa "probabilidade" de repetir mandato.


In [10]:
df_final = df_legislaturas_repetidos.merge(df_retribuciones_alcaldes, on="NOMBRE", how="inner")
df_final = df_final[df_final["TOTAL_PERCIBIDO"] > 0.00]
df_final.sort_values(by="TOTAL_PERCIBIDO", ascending=False) 

,NOMBRE,MUNICIPIO_x,FECHA_POSESION,REELEGIDO,TOTAL_PERCIBIDO,MUNICIPIO_y,COMUNIDAD_AUTONOMA_x,LISTA
10,ADA COLAU BALLANO,Barcelona,"[2015-06-13, 2019-06-15]",2,100000.04,Barcelona,Cataluna,C. ELECTORAL
2201,JUAN ESPADAS CEJAS,Sevilla,"[2015-06-13, 2019-06-15]",2,88673.68,Sevilla,Andalucia,PSOE
3325,RAMON FERNANDEZ-PACHECO MONTERREAL,Almeria,"[2015-11-28, 2019-06-15]",2,83376.86,Almeria,Andalucia,PP
1800,JOSE JULIAN MENA PEREZ,Arona,"[2015-06-13, 2019-06-15]",2,83054.37,Arona,Canarias,PSOE
3604,Susana Perez Quislant,Pozuelo de Alarcon,"[2015-06-13, 2019-06-15]",2,81919.13,Pozuelo de Alarcon,"Madrid, Comunidad de",PP
...,...,...,...,...,...,...,...,...
2208,JUAN FRANCISCO FIGUEROA RUIZ,Campillo de Arenas,"[2007-06-16, 2015-06-13, 2019-06-15]",3,50.00,Campillo de Arenas,Andalucia,PSOE
1813,JOSE LUCAS SANCHEZ,Galinduste,"[2007-06-16, 2011-06-11, 2015-06-13, 2019-06-15]",4,25.00,Galinduste,Castilla y Leon,PSOE
2641,MARCIAL GARCIA FERNANDEZ,Carmena,"[2014-06-26, 2015-06-13, 2019-06-15]",3,18.00,Carmena,Castilla-La Mancha,PSOE
1512,JOAN EROLES VILES,Preixens,"[2011-06-11, 2019-06-15]",2,1.00,Preixens,Cataluna,C. ELECTORAL


In [ ]:
#BONUS
# Agrupar por NOMBRE y contar ceses únicos
df_cesados_agrupados = df_cesados.groupby("NOMBRE").size().reset_index(name="MUNCIPIO_x")

df_cesados_agrupados = df_cesados_agrupados.sort_values(by="MUNICIPIOS_x", ascending=False)
df_cesados_agrupados

,NOMBRE,MUNICIPIOS_CESADOS
3153,JOSE OLIVER AMENGUAL,3
1873,FRANCISCO JAVIER PEREZ CUEVA,3
3360,JUAN ESCUDERO SANCHEZ,3
3477,JULIAN FELIPE MUNOZ PALOMO,3
5808,SEBASTIA PUJOL PUIG,3
...,...,...
6323,ZEUS SERRANO SOLER,1
6324,"ZORNOZA RUIZ, FELIPE",1
6325,"ZUBIZARRETA LASAGABASTER, XABIER",1
6326,"ZUMEL REPISO, JESUS ANGEL",1
